## Predicting Stress and Sleep Disorders Using Health and Lifestyle Data



# Team :-
### Shanmukha Sri Krishna
### Bhupati Varun
### Vishnu Shreeram

In [ ]:
import pandas as pd
import numpy as np

# Loading Dataset
To load the dataset, we can use the `read_csv()` function from the pandas library :

```
python
df_part1 = pd.read_csv('./Sleep_health_and_lifestyle_dataset.csv')
df_part2 = pd.read_csv('./Sleep_health_and_lifestyle_dataset_part_2.csv')
```
These two lines of code will load the dataset from the CSV files `Sleep_health_and_lifestyle_dataset.csv` and `Sleep_health_and_lifestyle_dataset_part_2.csv` respectively and store them in the variables `df_part1` and `df_part2`.


In [ ]:
df_part1 = pd.read_csv('./Sleep_health_and_lifestyle_dataset.csv')
df_part2 = pd.read_csv('./Sleep_health_and_lifestyle_dataset_part_2.csv')

# Merging two parts of the dataset to create a single one

In [ ]:
df = pd.concat((df_part1, df_part2))
print(df.shape)
df.head()

## Exploring the data set

In [ ]:
df.info()

In [ ]:
df.describe()

# Dropping Irrelevant Feature Person ID

In [ ]:
df.drop('Person ID', axis=1, inplace=True)

# Handling NA values  

In [ ]:
df.isnull().sum()

### NA values found in 'Sleep Disorder' indicates that the person doesn't have any sleep disorders, so instead of treating it as NA value we will assign 'None' to it

In [ ]:
df['Sleep Disorder'] = df['Sleep Disorder'].fillna('None')

In [ ]:
df.isnull().sum()

In [ ]:
df

## Handling the 'Blood Pressure' column by splitting it into Diastolic Pressure and Systolic Pressure

In [ ]:
lowRow = []
highRow = []


for val in df['Blood Pressure']:
    sp = val.split('/')

    highRow.append(int(sp[0]))
    lowRow.append(int(sp[1]))

df['Diastolic Pressure'] = np.array(lowRow)
df['Systolic Pressure'] = np.array(highRow)


In [ ]:
df = df.drop('Blood Pressure', axis = 1)

In [ ]:
df

## Using LabelEncoders to Encode the Categorical Data
#### We also store a dictionay of encoders so that we can also Transform the unseen data into the same format

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
catData = [key for key in df if (df[key].dtype == object)]
numData = [key for key in df if (df[key].dtype != object and key != 'Stress Level')]
print('categorical features are :',catData)

In [ ]:
label_encoding_dict = {}
for category in catData:
    labelEncoder = LabelEncoder()
    labelEncoder.fit(df[category])
    df[category] = labelEncoder.transform(df[category])
    label_encoding_dict[category] = labelEncoder
df

# Distribution of target features

In [ ]:
import matplotlib.pyplot as plt
count=df['Sleep Disorder'].value_counts()
plt.bar(label_encoding_dict['Sleep Disorder'].inverse_transform(count.keys()),count)
plt.xlabel('Sleep Disorder')
plt.ylabel('Number of people')
plt.show()

# Boxplot of Features

In [ ]:
import matplotlib.pyplot as plt

columns = ['Gender', 'Age', 'Occupation', 'Sleep Duration', 'Quality of Sleep', 'Physical Activity Level', 'BMI Category', 'Heart Rate', 'Daily Steps', 'Diastolic Pressure', 'Systolic Pressure']

fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(15, 10))

for i, column in enumerate(columns):
    ax = axes[i // 4, i % 4]
    ax.boxplot(df[column])
    ax.set_title(column)

plt.tight_layout()
plt.show()

## Correlation analysis

In [ ]:
import seaborn as sns

import matplotlib.pyplot as plt

# Calculate the correlation matrix
corr_matrix = df.corr()

# Plot the correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap of Features')
plt.show()

The correlation heatmap provides insights into the relationships between different features in the dataset
From the above heatmap, we can see that :-
- Systolic and Diastolic Pressures are highly correlated (+ve)
- Quality of Sleep and Sleep duration (+ve )
- Stress level and Quality of sleep / Sleep duaration (-ve)
- Daily steps and physical activity level (+ve)

# Standardisation of Data

In [ ]:
numData

In [ ]:
# Standardize the data
from sklearn.preprocessing import StandardScaler
for f in numData:
    df[f] = StandardScaler().fit_transform(df[[f]])
df

# Building Regression models for Stress level Prediction

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Regression
tgt1 = df['Stress Level']
feat1 = df.drop(columns = ['Stress Level','Sleep Disorder'])


In [ ]:
feat1

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
scoreList = []

for i in range(1, 5):

    poly = PolynomialFeatures(i)
    x = poly.fit_transform(feat1)

    X_train, X_test, y_train, y_test = train_test_split(x, np.array(tgt1), random_state = 42)

    model = LinearRegression()
    model.fit(X_train,y_train)
    pred = model.predict(X_test)

    scoreList.append(r2_score(y_test, pred))

scoreList

In [ ]:
xList = [i for i in range(1, 5)]

plt.xlabel("Degree of polynomial feature")
plt.ylabel("R2 Score")
plt.title("Graph of Degree vs R2 Score")
plt.plot(xList, scoreList)
plt.show()

# From the above plot, we select the degree of polynomial features as 3

In [ ]:
poly = PolynomialFeatures(3)
x = poly.fit_transform(feat1)

X_train, X_test, y_train, y_test = train_test_split(x, np.array(tgt1), random_state = 42)

model = LinearRegression()
model.fit(X_train,y_train)
pred = model.predict(X_test)
print('r2_score =',r2_score(y_test, pred))
print('mean_squared_error =',mean_squared_error(y_test, pred))

## Classification

In [ ]:
X = df.drop(columns=['Stress Level','Sleep Disorder'])
y = df['Sleep Disorder']

# Building Models for Classification of Sleep Disorders

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
clf = SVC(kernel ='linear')
 # training set in x, y axis
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('Classification Report',classification_report(y_test,y_pred))

In [ ]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('Classification Report',classification_report(y_test,y_pred))

#Using over sampling method SMOTE to balance the data

In [ ]:
from collections import Counter
cc = Counter(y_train)

# Checking for imbalanced data
cc

In [ ]:
# Bad news, we actually have a large amount of imbalanced data
# So using a imbalanced learn method helps us to predict good observations
# Using SMOTE: Synthetic Minority Oversampling Technique

from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state = 42)
X_smote, y_smote = sm.fit_resample(X_train, y_train)


In [ ]:
# Checking for balanced data in new target
cc = Counter(y_smote)
cc

## Using Undersampling Method

In [ ]:
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=0)
X_clc, y_clc = cc.fit_resample(X_train, y_train)


In [ ]:
# from imblearn.under_sampling import InstanceHardnessThreshold

# clc = InstanceHardnessThreshold()
# X_clc, y_clc = clc.fit_resample(X_train, y_train)

In [ ]:
cc = Counter(y_clc)
cc

# Using Randomized search CV

## Intial Data

In [ ]:
from scipy.stats import uniform

gammaValues = ['scale', 'auto']

cvals = [1e-5, 1e-4, 1e-3, 0.01, 0.1, 1, 10]

param_grid = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': uniform(1e-5, 10),
    'degree': [i for i in range(1, 20)],
    'gamma': gammaValues,
    'decision_function_shape': ['ovo', 'ovr']
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

base_clf = SVC()
rand_clf = RandomizedSearchCV(base_clf, param_distributions = param_grid)

rand_clf.fit(X_train, y_train)

In [ ]:
rand_clf.best_params_

In [ ]:
# Printing the classification report for the randomized search hyper parameter tuned model

y_pred = rand_clf.predict(X_test)

print("The classification report after using randomized search CV\n", classification_report(y_test, y_pred))

In [ ]:
# Printing the confusion matrix for the randomized search hyper parameter tuned model
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

# Randomized search CV for SMOTE

In [ ]:
base_clf = SVC()
rand_clf = RandomizedSearchCV(base_clf, param_distributions = param_grid)

rand_clf.fit(X_clc, y_clc)

In [ ]:
rand_clf.best_params_

In [ ]:
# Printing the classification report for the randomized search hyper parameter tuned model

y_pred = rand_clf.predict(X_test)

print("The classification report after using randomized search CV\n", classification_report(y_test, y_pred))

In [ ]:
confusion_matrix(y_test, y_pred)

# RandomizedSearchCV for Under Sampled Data

In [ ]:
base_clf = SVC()
rand_clf = RandomizedSearchCV(base_clf, param_distributions = param_grid, verbose = 3)

rand_clf.fit(X_smote, y_smote)

In [ ]:
rand_clf.best_params_

In [ ]:
y_pred = rand_clf.predict(X_test)

print("The classification report after using randomized search CV\n", classification_report(y_test, y_pred))

In [ ]:
confusion_matrix(y_test, y_pred)

#Using GridSearchCV for hyper parameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

gammaValues = ['scale', 'auto']

cvals = [1e-5, 1e-4, 1e-3, 0.01, 0.1, 1, 10]

param_grid = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': cvals,
    'degree': [i for i in range(1, 5)],
    'gamma': gammaValues,
    'decision_function_shape': ['ovo', 'ovr']
}

base_clf = SVC()
grid_clf = GridSearchCV(base_clf, param_grid = param_grid, verbose = 3)

grid_clf.fit(X_train, y_train)


In [ ]:
# Finding the best parameters for the gridsearch CV
grid_clf.best_params_

In [ ]:
# Using the test data to make predictions on the input
y_pred = grid_clf.predict(X_test)

In [ ]:
print('Classification Report after using grid search CV\n', classification_report(y_pred = y_pred, y_true = y_test))

In [ ]:


print("The confusion matrix is as follows")
confusion_matrix(y_test, y_pred)

## Using SMOTE samples for GridSearchCV


In [ ]:
base_clf = SVC()
grid_clf = GridSearchCV(base_clf, param_grid = param_grid, verbose = 3)

grid_clf.fit(X_smote, y_smote)

In [ ]:
grid_clf.best_params_

In [ ]:
y_pred = grid_clf.predict(X_test)

print('Classification Report after using grid search CV on SMOTE data\n', classification_report(y_pred = y_pred, y_true = y_test))

In [ ]:
confusion_matrix(y_pred = y_pred, y_true = y_test)

# Using GridSearchCV for Under Sampled data

In [ ]:
param_grid = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': cvals,
    'degree': [i for i in range(1, 5)],
    'gamma': gammaValues,
    'decision_function_shape': ['ovo', 'ovr']
}

In [ ]:
base_clf = SVC()
grid_clf = GridSearchCV(base_clf, param_grid = param_grid, verbose = 3)

grid_clf.fit(X_clc, y_clc)

In [ ]:
grid_clf.best_params_

In [ ]:
y_pred = grid_clf.predict(X_test)

print('Classification Report after using grid search CV on SMOTE data\n', classification_report(y_pred = y_pred, y_true = y_test))

In [ ]:
confusion_matrix(y_true = y_test, y_pred = y_pred)

# Visualising clusters using Principal Component Analysis (PCA and Kernel PCA)

In [ ]:
from sklearn.decomposition import PCA
label_encoding_dict['Sleep Disorder'].classes_

In [ ]:
features = [key for key in df if (key != 'Sleep Disorder' and key != 'Stress Level')]
# ['Insomnia', 'None', 'Sleep Apnea'] the labels associated with 0,1,2
pca = PCA(n_components=len(features))
df_full_pca = pca.fit_transform(df[features])
labels = label_encoding_dict['Sleep Disorder'].classes_
colors = ['r', 'g', 'b']

for i, label in enumerate(labels):
    plt.scatter(df_full_pca[df['Sleep Disorder'] == i, 0],
                df_full_pca[df['Sleep Disorder'] == i, 1],
                label=label,
                color=colors[i])
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA of Sleep Disorder')
plt.legend()
plt.show()

## Visualising Explained Variance Ratio vs Number of Components, & finding the optimal number of components

In [ ]:
# explained variance ratio graph
plt.plot([i+1 for i in range(len(pca.explained_variance_ratio_)) ],np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained Variance Ratio vs Number of Components')
# show all the ticks in x-axis
plt.xticks([i+1 for i in range(len(pca.explained_variance_ratio_))])
# show the points in the graph with x,y values
for i in range(len(pca.explained_variance_ratio_)):
    plt.text(i+1, np.cumsum(pca.explained_variance_ratio_)[i], f'({i+1}, {np.cumsum(pca.explained_variance_ratio_)[i]:.2f})', fontsize=6
             , verticalalignment='bottom', horizontalalignment='right')
    plt.grid()
plt.show()

## From here we can see that at Number of Components = 5, it has achieved 91% of cumulative variance ratio. So we would try the classification using the new 5 components

In [ ]:
features = [key for key in df if (key != 'Sleep Disorder' and key != 'Stress Level')]
new_pca = PCA(n_components=5)
new_pca.fit(df[features])
new_points = new_pca.transform(df[features])

In [ ]:
X = df.drop(columns=['Stress Level','Sleep Disorder'])
y = df['Sleep Disorder']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=42)

from sklearn.svm import SVC
from sklearn.metrics import classification_report

# {'C': 8.430136277531556,
#  'decision_function_shape': 'ovr',
#  'degree': 4,
#  'gamma': 'auto',
#  'kernel': 'rbf'}





clf = SVC(kernel='rbf',degree=4,C=8.430136277531556,gamma='auto',decision_function_shape='ovr') # found 11 through various trial

 # training set in x, y axis
clf.fit(new_pca.transform(X_train),y_train)
y_pred = clf.predict(new_pca.transform(X_test))

print('Classification Report',classification_report(y_test,y_pred))

#using ensemble methods for imporving accuracy and reduce overfitting


In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
y2 = df['Sleep Disorder']
y = df['Stress Level']

# Ensembles for Regression

In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
regr = BaggingRegressor(estimator=SVR(),
                        n_estimators=80, random_state=0).fit(X_train, y_train)
y_pred = regr.predict(X_test)

from sklearn.metrics import mean_squared_error
print('mean squared error' , mean_squared_error(y_test,y_pred))

In [ ]:

rf_regressor = RandomForestRegressor(random_state=42)
rf_regressor.fit(X_train, y_train)


y_pred_reg = rf_regressor.predict(X_test)
mse_reg = mean_squared_error(y_test, y_pred_reg)
r2_reg = r2_score(y_test, y_pred_reg)

# Calculate the differences (residuals)
residuals = y_test - y_pred_reg

# Plot the residuals
plt.figure(figsize=(10, 6))
plt.scatter(range(len(residuals)), y_test, label="Actual", color='blue', marker='o', alpha=0.7)
plt.scatter(range(len(residuals)), y_pred_reg, label="Predicted", color='red', marker='o', alpha=0.7)

plt.axhline(0, color='black', linestyle='--', lw=2)  # Add a horizontal line at y=0 for reference
plt.xlabel("Index")
plt.ylabel("Difference in Stress Level")
plt.title("Difference between Actual and Predicted Stress Level")
plt.legend()
plt.show()


print("mse_reg",mse_reg)
print("r2_reg",r2_reg)


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(len(residuals)), residuals, label="Predicted", color='purple', marker='o', alpha=0.6)
plt.xlabel("Index")
plt.ylabel("Difference in Stress Level")
plt.title("Difference between Actual and Predicted Stress Level")

plt.show()


# classification using Ensembles

In [ ]:
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X, y2, test_size=0.2, random_state=42)


In [ ]:
# 3. Train a Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train_clf, y_train_clf)

# 4. Predict on the test set
y_pred_clf = rf_classifier.predict(X_test_clf)

# 5. Calculate classification metrics
accuracy_clf = accuracy_score(y_test_clf, y_pred_clf)

# 6. Confusion matrix
conf_matrix = confusion_matrix(y_test_clf, y_pred_clf)
ConfusionMatrixDisplay(conf_matrix).plot(cmap="Blues")
plt.title("Confusion Matrix for Sleep Disorder Classification")
plt.show()

print(f"Classification Accuracy: {accuracy_clf}")
print('confusion_matrix \n',conf_matrix)
# Explanation of Corrections:

## Finding the best n_estimators for Random Forest Classifier


In [ ]:
l2 = []
for  i in range(1,80):
    rf_classifier = RandomForestClassifier(n_estimators=i,random_state=42)
    rf_classifier.fit(X_train_clf, y_train_clf)

    # 4. Predict on the test set
    y_pred_clf = rf_classifier.predict(X_test_clf)

    # 5. Calculate classification metrics
    accuracy_clf = accuracy_score(y_test_clf, y_pred_clf)
    l2.append(accuracy_clf)
plt.plot(range(1,80),l2)
plt.ylabel('Accuracy')
plt.xlabel('Number of Estimators')
plt.show()

In [ ]:
# choosing various ensemble methods for regression and classification

In [ ]:

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor

from xgboost import XGBRegressor, XGBClassifier


# Split the data into training and testing sets
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y, test_size=0.2, random_state=42)

# Define and train ensemble regressors
regressors = {
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "AdaBoost": AdaBoostRegressor(random_state=42),
    "Bagging": BaggingRegressor(estimator=RandomForestRegressor(), random_state=42),
    "XGBoost": XGBRegressor(objective='reg:squarederror', random_state=42)
}

# Evaluate each regressor
for name, model in regressors.items():
    model.fit(X_train_reg, y_train_reg)
    y_pred_reg = model.predict(X_test_reg)
    mse = mean_squared_error(y_test_reg, y_pred_reg)
    r2 = r2_score(y_test_reg, y_pred_reg)
    print(f"{name} Regressor - MSE: {mse:.4f}, R2 Score: {r2:.4f}")
    residuals = y_test_reg - y_pred_reg

    # Plot the residuals
    plt.figure(figsize=(10, 6))
    plt.plot(range(len(residuals)), residuals, label="Difference (Actual - Predicted)", color='purple', marker='o', alpha=0.7)
    plt.axhline(0, color='black', linestyle='--', lw=2)  # Add a horizontal line at y=0 for reference
    plt.xlabel("Index")
    plt.ylabel("Difference in Stress Level")
    plt.title(f"Difference between Actual and Predicted Stress Level {name}")
    plt.legend()
    plt.show()


### The best Ensemble Regressor is XG Boost

In [ ]:


from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X, y2, test_size=0.2, random_state=42)

# Define and train ensemble classifiers
classifiers = {
    "Random Forest": RandomForestClassifier(n_estimators=40,random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "Bagging": BaggingClassifier(estimator=RandomForestClassifier(), random_state=42),
    "XGBoost": XGBClassifier(eval_metric='logloss', random_state=42)
}

# Evaluate each classifier
for name, model in classifiers.items():
    model.fit(X_train_clf, y_train_clf)
    y_pred_clf = model.predict(X_test_clf)
    accuracy = accuracy_score(y_test_clf, y_pred_clf)
    print(f"{name} Classifier - Accuracy: {accuracy:.4f}")

    # Confusion Matrix for each classifier
    conf_matrix = confusion_matrix(y_test_clf, y_pred_clf)
    ConfusionMatrixDisplay(conf_matrix).plot(cmap="Blues")
    plt.title(f"Confusion Matrix for {name} Classifier")
    plt.show()

## The best Ensemble Classifier are Random Forest, Gradient Boosting and XG Boost

# precision recall graphs


In [ ]:
from sklearn.preprocessing import label_binarize
y=df['Sleep Disorder']
# Use label_binarize to be multi-label like settings
Y = label_binarize(y, classes=[0, 1, 2])
n_classes = Y.shape[1]

# Split into training and test
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import make_pipeline
classifier = OneVsRestClassifier(
    make_pipeline(StandardScaler(), SVC(random_state= 42))
)
classifier.fit(X_train2, Y_train2)
y_score = classifier.decision_function(X_test2)

from sklearn.metrics import average_precision_score, precision_recall_curve

# For each class
precision = dict()
recall = dict()
average_precision = dict()
for i in range(3):
    precision[i], recall[i], _ = precision_recall_curve(Y_test2[:, i] , y_score[:, i])
    average_precision[i] = average_precision_score(Y_test2[:, i] , y_score[:, i])

# A "micro-average": quantifying score on all classes jointly
precision["micro"], recall["micro"], _ = precision_recall_curve(
    Y_test2.ravel(), y_score.ravel()
)
average_precision["micro"] = average_precision_score(Y_test2 , y_score, average="micro")

from collections import Counter
from sklearn.metrics import PrecisionRecallDisplay
display = PrecisionRecallDisplay(
    recall=recall["micro"],
    precision=precision["micro"],
    average_precision=average_precision["micro"],
    prevalence_pos_label=Counter(Y_test2.ravel())[1] / Y_test2.size,
)
display.plot(plot_chance_level=True)
_ = display.ax_.set_title("Micro-averaged over all classes")

In [ ]:
from itertools import cycle

import matplotlib.pyplot as plt

# setup plot details
colors = cycle(["navy", "turquoise", "darkorange", "cornflowerblue", "teal"])

_, ax = plt.subplots(figsize=(7, 8))

f_scores = np.linspace(0.2, 0.8, num=4)
lines, labels = [], []
for f_score in f_scores:
    x = np.linspace(0.01, 1)
    y = f_score * x / (2 * x - f_score)
    (l,) = plt.plot(x[y >= 0], y[y >= 0], color="gray", alpha=0.2)
    plt.annotate("f1={0:0.1f}".format(f_score), xy=(0.9, y[45] + 0.02))

display = PrecisionRecallDisplay(
    recall=recall["micro"],
    precision=precision["micro"],
    average_precision=average_precision["micro"],
)
display.plot(ax=ax, name="Micro-average precision-recall", color="gold")

for i, color in zip(range(n_classes), colors):
    display = PrecisionRecallDisplay(
        recall=recall[i],
        precision=precision[i],
        average_precision=average_precision[i],
    )
    display.plot(ax=ax, name=f"Precision-recall for class {i}", color=color)

# add the legend for the iso-f1 curves
handles, labels = display.ax_.get_legend_handles_labels()
handles.extend([l])
labels.extend(["iso-f1 curves"])
# set the legend and the axes
ax.legend(handles=handles, labels=labels, loc="best")
ax.set_title("Extension of Precision-Recall curve to multi-class")

plt.show()